In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from twilio.rest import Client
from dotenv import load_dotenv
import os
from pathlib import Path



PROJECT_ROOT = Path(r"D:\Jupyter_notebook\GitHub\AI_lunch_box_recommender_system")
ENV_PATH = PROJECT_ROOT / ".env"

load_dotenv(ENV_PATH)

# sanity check (run once)
print("OPENAI_API_KEY loaded:", "OPENAI_API_KEY" in os.environ)

In [8]:

# Global variables
api_key=os.environ["OPENAI_API_KEY"]
if not api_key:
    raise RuntimeError("OPENAI_API_KEY is missing. Did you create and load the .env file?")
client = OpenAI(api_key=api_key)

                
url = "https://www.ica.se/erbjudanden/ica-supermarket-aptiten-1003988/"

client_twilio = Client(
    os.environ["account_sid"], 
    os.environ["auth_token"]
)

def scrape_your_local_ica(url):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)

    # clicking agree on cookie button
    agree_button_selector = "button#onetrust-accept-btn-handler"
    WebDriverWait(driver, 3)
    element = driver.find_element(By.CSS_SELECTOR, agree_button_selector)
    element.click()

    # scroll to bottom of page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # generate a soup
    soup = BeautifulSoup(driver.page_source)
    key = "article > div.offer-card__details-container"
    containers = soup.select(key)
    offers = [i.text.replace("\xa0kr. ", "") for i in containers]
    offers = [i.split(". ")[:2] for i in offers]
    return offers

def send_text(from_number, to_number, text_message):
    message = client_twilio.messages.create(
        body=text_message,
        from_=from_number,
        to=to_number
    )

def chat_gpt_bot(client, instructions, prompt):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": prompt},
        ]
    )
    return completion.choices[0].message.content

role = "Du är en kreativ kock som är väldigt duktig på att rekommendera enkla maträtter baserat på tillgängliga ingredienser."
instructions = "Jag vill att du rekommenderar mig maträtter som jag kan laga till middag och sedan göra matlåda av. Du får en lista med veckans rabatterade priser från en matbutik. Du ska använda listan för att skapa 3 enkla recept. Du får också lägga till basvaror (pasta, ris, potatis, lök etc.), även om de inte är med på listan.\nJag vill att du genererar 3 rätter i rubrikform och under varje rubrik/rätt så ska det finnas en lista med ingredienser för 4 portioner. Det är viktigt att enheterna är i enlighet med det metriska systemet. Under listan med ingredienser ska du beskriva väldigt kort hur man tillagar rätten."



# scrape ICAs website
context = scrape_your_local_ica(url)
completion = chat_gpt_bot(client,role,instructions + str(context))
shopping_lst = chat_gpt_bot(client, "Hjälpreda", "Generera en inköpslista i punktform med sammanlagda enheter av följande information: "+completion)

#send_text(os.environ["from_number"],os.environ["to_number"],shopping_lst)

# write to recipes to txt file
with open("weekly_recipes.txt","w", encoding="utf-8") as file:
    file.write(completion)
    file.close()

# write to shopping list to file (if twilio doesn't work for you)
with open("shopping_lst.txt","w", encoding="utf-8") as file:
    file.write(shopping_lst)
    file.close()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [5]:
import sys
sys.executable

'C:\\Users\\taoti\\anaconda3\\python.exe'

In [7]:
%pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install selenium requests beautifulsoup4 openai twilio python-dotenv



   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ----------- ---------------------------- 0.3/1.1 MB 6.5 MB/s eta 0:00:01
   ------------------------ --------------- 0.6/1.1 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------  1.0/1.1 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ------ --------------------------------- 0.4/2.3 MB 12.2 MB/s eta 0:00:01
   ----------- ---------------------------- 0.6/2.3 MB 8.0 MB/s eta 0:00:01
   --------------- ------------------------ 0.8/2.3 MB 6.7 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.3 MB 7.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.3 MB 7.0 MB/s eta 0:00:01
   ------------------------------------ --- 2.1/2.3 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 7.2 MB/s eta 0:00:00
   ----------------------